# Training Convergence Plots for Learning Punctuation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Serif'
plt.rcParams['font.serif'] = 'Liberation Serif'
import pickle
import os
print(os.getcwd())
import time
import gc
import torch
import bisect

from conv_plots_funcs import train_conv_plots

#import empirical cdf
# import sys
# sys.path.append(os.path.abspath('../../src'))

from data_processing import gen_ckpt_steps, move_dict_to_device, get_other_err, get_mop_ratios_ckpt, compute_ratio
# sys.path.append(os.path.abspath('..'))

from check_ecdf import get_empirical_cdf
from pretrain_loss import compute_pseudo_pred_avg_pipeline, get_multi_sys_ys, compute_pseudo_pred_errs, compute_pseudo_pred_errs_needle_in_haystack

In [ ]:
valA = "ortho"
valC = "_ident_C"
state_dim = 5
ckpt_step = 8
batch_size = 512 #512 #4096
gpus=2
num_val_sys = 25
experiment = "250112_043028.07172b_multi_sys_trace_ortho_state_dim_5_ident_C_lr_1.584893192461114e-05_num_train_sys_40000" # experiment to load
datasource = "val"
nope = False
single_system = True
zero_cut = False


compute_more_ckpts = False

In [ ]:
kal_ckpt = 141000 #60000 #81000 #28500
train_conv_plots([experiment], ["Identity"], [kal_ckpt], valA, valC, num_val_sys, compute_more_ckpts, None, 3000, 180000, 3000, state_dim, single_system=single_system, nope=nope, batch_size=batch_size, gpus=gpus, zero_cut=zero_cut)

# Zero Cut Plot

In [ ]:
valA = "gaussA"
valC = "_gauss_C"
batch_size = 512 #512 #4096
gpus=2
num_val_sys = 25
experiment_zero_train = "250127_001511.3ac954_multi_sys_trace_zero_cut_gaussA_state_dim_10_gauss_C_lr_1.584893192461114e-05_num_train_sys_40000" # experiment to load
experiment_multi_train = "250114_202420.3c1184_multi_sys_trace_gaussA_state_dim_10_gauss_C_lr_1.584893192461114e-05_num_train_sys_40000"
kal_ckpt_zero_train = 90000 
# kal_ckpt_multi_train = 30000 #180000
datasource = "val"
state_dim = 10

In [ ]:
kal_err_zero_train = get_other_err(valA, valC, kal_ckpt_zero_train, experiment_zero_train, "Kalman_rem", state_dim, zero_cut=True)

tf_err_zero_train, pred_ckpt = get_mop_ratios_ckpt(valA, valC, kal_ckpt_zero_train, experiment_zero_train, state_dim, zero_cut=True)

ols_irs = []
for ir in range(1, 4):
    ols_ir = get_other_err(valA, valC, kal_ckpt_zero_train, experiment_zero_train, f"OLS_ir_{ir}", state_dim, zero_cut=True)
    ols_irs.append(compute_ratio(None, ols_ir, kal_err_zero_train).cpu().numpy())

tf_err_rat_zero_train = compute_ratio(None, tf_err_zero_train, kal_err_zero_train)
tf_err_rat_zero_train = tf_err_rat_zero_train.cpu().numpy()
print("shape of tf_err_rat_zero_train: ", tf_err_rat_zero_train.shape)

In [ ]:
#save zero train data
os.makedirs("../outputs/GPT2/" + experiment_zero_train + "/context", exist_ok=True)
np.save("../outputs/GPT2/" + experiment_zero_train + f"/context/tf_err_rat_zero_train_step_{kal_ckpt_zero_train}.npy", tf_err_rat_zero_train)
np.save("../outputs/GPT2/" + experiment_zero_train + "/context/ols_irs_zero_train.npy", ols_irs)

In [ ]:
experiment_multi_train = "250418_125901.8d6b22_multi_sys_trace_ortho_haar_state_dim_5_ident_C_lr_1.584893192461114e-05_num_train_sys_40000"
valA = "ortho_haar"
valC = "_ident_C"
state_dim = 5
batch_size = 512
gpus=1

max_ckpt = 183000


In [ ]:
for kal_ckpt_multi_train in np.arange(1000, max_ckpt, 1000):
    if kal_ckpt_multi_train < max_ckpt:
        train_conv = True
    else:
        train_conv = False

    tf_err_multi_train, pred_ckpt = get_mop_ratios_ckpt(valA, valC, kal_ckpt_multi_train, experiment_multi_train, state_dim, zero_cut=True, train_conv=train_conv)

    if tf_err_multi_train is None:
        print("No data for step: ", kal_ckpt_multi_train)
        continue
    # an_sim_err_multi_train = get_other_err(valA, valC, kal_ckpt_multi_train, experiment_multi_train, "Analytical_Simulation", state_dim, zero_cut=True, train_conv=train_conv)
    # an_sim_err_multi_train = compute_ratio(None, an_sim_err_multi_train, kal_err_zero_train).cpu().numpy()

    # ols_irs = []
    # for ir in range(1, 4):
    #     ols_ir = get_other_err(valA, valC, kal_ckpt_multi_train, experiment_zero_train, f"OLS_ir_{ir}", state_dim, zero_cut=True)
    #     ols_irs.append(compute_ratio(None, ols_ir, kal_err_zero_train).cpu().numpy())

    tf_err_rat_multi_train = compute_ratio(None, tf_err_multi_train, kalman_err=None)
    tf_err_rat_multi_train = tf_err_rat_multi_train.cpu().numpy()
    print("shape of tf_err_rat_multi_train: ", tf_err_rat_multi_train.shape)

    #save the data
    os.makedirs("../outputs/GPT2/" + experiment_multi_train + "/context", exist_ok=True)
    np.save("../outputs/GPT2/" + experiment_multi_train + f"/context/tf_err_rat_multi_train_step_{kal_ckpt_multi_train}.npy", tf_err_rat_multi_train)

#### load the data

In [ ]:
# tf_err_rat_zero_train = np.load("../outputs/GPT2/" + experiment_zero_train + "/context/tf_err_rat_zero_train.npy")
# ols_irs = np.load("../outputs/GPT2/" + experiment_zero_train + "/context/ols_irs_zero_train.npy")

# print("tf_err_rat_zero_train: ", tf_err_rat_zero_train)

ckpt_steps = []
tf_err_multi_trains = []
for kal_ckpt_multi_train in np.arange(1000, max_ckpt, 1000):

    try:
        tf_err_multi_train = np.load("../outputs/GPT2/" + experiment_multi_train + f"/context/tf_err_rat_multi_train_step_{kal_ckpt_multi_train}.npy")
        print(f"kal_ckpt_multi_train: {kal_ckpt_multi_train}")
        # print(f"kal_ckpt_multi_train: {kal_ckpt_multi_train}, tf_err_multi_train: {tf_err_multi_train}")
        tf_err_multi_trains.append(tf_err_multi_train)
    except FileNotFoundError:
        print(f"File not found for kal_ckpt_multi_train: {kal_ckpt_multi_train}")
        continue

    ckpt_steps.append(kal_ckpt_multi_train)

# tf_err_rat_multi_train_3k = np.load("../outputs/GPT2/" + experiment_multi_train + "/context/tf_err_rat_multi_train_step_3000.npy")
# tf_err_rat_multi_train_30k = np.load("../outputs/GPT2/" + experiment_multi_train + "/context/tf_err_rat_multi_train_step_30000.npy")
# tf_err_rat_multi_train_96k = np.load("../outputs/GPT2/" + experiment_multi_train + "/context/tf_err_rat_multi_train_step_96000.npy")
# tf_err_rat_multi_train_180k = np.load("../outputs/GPT2/" + experiment_multi_train + "/context/tf_err_rat_multi_train_step_180000.npy")



In [ ]:
print("batch_size: ", batch_size)
print("gpus: ", gpus)
print("batch_size*kal_ckpt_multi_train: ", batch_size*kal_ckpt_multi_train)

In [ ]:
tf_err_rat_zero_train -= 1
for ir in range(3):
    ols_irs[ir] -= 1

for i in range(len(tf_err_multi_trains)):
    tf_err_multi_trains[i] -= 1

In [ ]:
def scientific_notation(value):
    """
    Returns the exponent and value of the scientific notation of an integer.
    
    Parameters:
    value (int): The integer to convert to scientific notation.
    
    Returns:
    tuple: A tuple containing the value and exponent of the scientific notation.
    """
    if value == 0:
        return (0, 0)
    
    exponent = 0
    while abs(value) >= 10:
        value /= 10
        exponent += 1
    while abs(value) < 1:
        value *= 10
        exponent -= 1

    value = round(value, 2)
    
    return (value, exponent)

### pseudo predictor

In [ ]:
from data_train import generate_interleaved_traces

In [ ]:
datasource = "train"
multi_sys_ys, sys_choices_per_config, seg_starts_per_config, sys_inds_per_config, real_seg_lens_per_config, sys_dict_per_config = get_multi_sys_ys(datasource, hay_len=1, interleave_style="needle_in_haystack")

pseudo_pred_errs = compute_pseudo_pred_errs_needle_in_haystack(multi_sys_ys, seg_starts_per_config, real_seg_lens_per_config, sys_choices_per_config, zero_cut=True)

In [ ]:
print(f"sum of pseudo_pred_errs: {np.sum(pseudo_pred_errs)}")
print(f"pseudo_pred_errs[0,0,0]: {pseudo_pred_errs[39000,0,6]}")

In [ ]:
#save the pseudo_pred_errs to a file
os.makedirs(f"/data/shared/ICL_Kalman_Experiments/train_and_test_data/ortho_haar/", exist_ok=True)
with open(f"/data/shared/ICL_Kalman_Experiments/train_and_test_data/ortho_haar/train_irrelevant_tokens_new_hay_insert_pseudo_pred_errs_ortho_haar_ident_C_haystack_len_1.pkl", 'wb') as f:
    pickle.dump(pseudo_pred_errs, f)
    print(f"Saved pseudo_pred_errs")

In [ ]:
#load the pseudo_pred_errs from a file
with open(f"/data/shared/ICL_Kalman_Experiments/train_and_test_data/ortho_haar/val_zero_cut_pseudo_pred_errs_ortho_haar_ident_C_haystack_len_.pkl", 'rb') as f:
    pseudo_pred_errs = pickle.load(f)
    print(f"Loaded pseudo_pred_errs")

In [ ]:
from matplotlib.colors import BoundaryNorm, ListedColormap

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
context_len = 251

# colors = ['#000000', '#005CAB', '#E31B23', '#FFC325', '#00A651', '#9B59B6', '#FF69B4', '#8B4513']

#make a colormap for len(tf_err_multi_trains) + 4 colors
# colors = plt.cm.viridis(np.linspace(0, 1, len(tf_err_multi_trains) + 5))
# print(np.linspace(0, 1, len(tf_err_multi_trains) + 5))
# intervals = np.linspace(0, 60, len(tf_err_multi_trains))
# intervals = (0.97)**(intervals)
# # intervals -= intervals.min()
# # intervals = 1 - intervals
# #reverse the intervals
# intervals = intervals[::-1]
# print(intervals)
# start = 0.01
# stop = 0.99
# eps = 0
# number_of_lines=len(tf_err_multi_trains)
# inter = start
# intervals = []
# for i in range(number_of_lines):
#     intervals.append(inter)
#     inter += eps
#     if i % 25 == 0:
#         eps += 0.015

# #add 1 to every element in intervals
# # intervals = np.array(intervals)
# # intervals -= 0.2
# # intervals = intervals/np.max(intervals)
# # print(intervals)

frac = 0.4
intervals = np.linspace(0,1, int(frac*len(tf_err_multi_trains))+1)
intervals = np.array(intervals)
intervals += 0.4
print(intervals)
print(len(intervals))

cmap = plt.cm.Greys
# norm = BoundaryNorm(intervals, cmap.N)


colors_1 = cmap(intervals)

intervals = np.linspace(0,1, int((1-frac)*len(tf_err_multi_trains))+1)
print(intervals)

cmap2 = plt.cm.magma
# norm2 = BoundaryNorm(intervals, cmap2.N)
colors_2 = cmap2(intervals)


combined_colors = np.vstack((colors_1, colors_2))

#add a single dark neon blue color to the middle of the combined colors
add_color = np.array([0.0, 0.5, 1.0, 1.0])


# After you have filled examples_list:
target_examples = 6.25e7

# multiply all elements of ckpt_steps by batch_size*gpus to get examples_list_orig
examples_list_orig = np.array(ckpt_steps) * batch_size * gpus

# Find the index where examples_list is closest to target_examples
insert_idx = np.argmin(np.abs(np.array(examples_list_orig) - target_examples))


# Insert the dark blue color at the correct position
combined_colors = np.insert(combined_colors, insert_idx, add_color, axis=0)

combined_cmap = ListedColormap(combined_colors)

combined_intervals = np.linspace(0, 1, len(combined_colors))


# zero_steps = scientific_notation(2048*kal_ckpt_zero_train)
# steps_180k = scientific_notation(180000*batch_size*gpus)
# steps_96k = scientific_notation(96000*batch_size*gpus)
# steps_30k = scientific_notation(30000*batch_size*gpus)
# steps_3k = scientific_notation(3000*batch_size*gpus)

# ax.plot(np.arange(context_len), tf_err_rat_zero_train[1], label=f"Zero-cut Trained at {zero_steps[0]}e{zero_steps[1]} examples", linewidth=2, marker="x", markersize=2, zorder=5, color = colors[1])#linestyle="--")
# ax.fill_between(np.arange(context_len), tf_err_rat_zero_train[0], tf_err_rat_zero_train[2], alpha=0.2, color = colors[1])

# ax.plot(np.arange(context_len), tf_err_rat_multi_train_180k[1], label=f"Multi-cut Trained at {steps_180k[0]}e{steps_180k[1]} examples", linewidth=2, marker="x", markersize=2, zorder=4, color=colors[0])#, linestyle="-.")
# ax.fill_between(np.arange(context_len), tf_err_rat_multi_train_180k[0], tf_err_rat_multi_train_180k[2], alpha=0.2, color=colors[0])

# ax.plot(np.arange(context_len), tf_err_rat_multi_train_96k[1], label=f"Multi-cut Trained at {steps_96k[0]}e{steps_96k[1]} examples", linewidth=2, marker="x", markersize=2, zorder=3, color=colors[5])#, linestyle="-.")
# ax.fill_between(np.arange(context_len), tf_err_rat_multi_train_96k[0], tf_err_rat_multi_train_96k[2], alpha=0.2, color=colors[5])

# ax.plot(np.arange(context_len), tf_err_rat_multi_train_30k[1], label=f"Multi-cut Trained at {steps_30k[0]}e{steps_30k[1]} examples", linewidth=2, marker="x", markersize=2, zorder=2, color=colors[7])#, linestyle="-.")
# ax.fill_between(np.arange(context_len), tf_err_rat_multi_train_30k[0], tf_err_rat_multi_train_30k[2], alpha=0.2, color=colors[7])


# ax.plot(np.arange(context_len), tf_err_rat_multi_train_3k[1], label=f"Multi-cut Trained at {steps_3k[0]}e{steps_3k[1]} examples", linewidth=2, marker="x", markersize=2, zorder=2, color=colors[6])#, linestyle="-.")
# ax.fill_between(np.arange(context_len), tf_err_rat_multi_train_3k[0], tf_err_rat_multi_train_3k[2], alpha=0.2, color=colors[6])

# ax.plot(np.arange(context_len), an_sim_err_multi_train[1], label="Analytical Simulation", linewidth=2, marker=".", markersize=10)
# ax.fill_between(np.arange(context_len), an_sim_err_multi_train[0], an_sim_err_multi_train[2], alpha=0.15)

# for ir in range(1, 4):
#     ax.plot(np.arange(context_len), ols_irs[ir-1][1], label=f"OLS-{ir}", linewidth=2, marker=".", markersize=2, color=colors[ir+1])
#     ax.fill_between(np.arange(context_len), ols_irs[ir-1][0], ols_irs[ir-1][2], alpha=0.2, color=colors[ir+1])

# steps = np.arange(3000, 183000, 3000)
# examples_list = []
# colors = colors_1
# color_count = 0
# for i in range(len(tf_err_multi_trains)):
#     print(f"i: {i}, color_count: {color_count}")
#     if i > frac*len(tf_err_multi_trains):
#         colors = colors_2
#         if color_count == i:
#             color_count = 0
#     examples = steps[i]*batch_size*gpus
#     examples_list.append(examples)
#     examples = scientific_notation(examples)
#     ax.plot(np.arange(context_len), tf_err_multi_trains[i][1], label=f"Multi-cut Trained at {examples[0]}e{examples[1]} examples", linewidth=2, marker="x", markersize=2, color=colors[color_count])
#     color_count += 1


# steps = np.arange(1000, max_ckpt, 1000)
examples_list = []
color_count = 0
for i in range(len(tf_err_multi_trains)):
    print(f"i: {i}, color_count: {color_count}")
    examples = ckpt_steps[i]*batch_size*gpus
    examples_list.append(examples)
    examples = scientific_notation(examples)
    ax.plot(np.arange(context_len), tf_err_multi_trains[i][1], label="_no_legend_", linewidth=2, marker=".", markersize=2, color=combined_colors[color_count])
    if i == insert_idx:

        fig.text(0.66,0.17, f"{examples[0]}e{examples[1]} ex", fontweight=10)
    color_count += 1

    # ax.fill_between(np.arange(context_len), tf_err_multi_trains[i][0], tf_err_multi_trains[i][2], alpha=0.2, color=colors[i+5])

#plot the pseudo prediction errors
#take the median of the pseudo prediction errors
pseudo_pred_meds = np.median(pseudo_pred_errs, axis=1)
print(f"pseudo_pred_meds[0,2]: {pseudo_pred_meds[0,2]}")
print(f"shape of pseudo_pred_meds: {pseudo_pred_meds.shape}")
pseudo_pred_qs = np.quantile(pseudo_pred_meds, [0.25, 0.5, 0.75], axis=0)
print(f"shape of pseudo_pred_qs: {pseudo_pred_qs.shape}")
print(f"pseudo_pred_qs: {pseudo_pred_qs[1]}")
# ax.plot(np.arange(2, context_len), pseudo_pred_qs[1,2:], label="Pseudoinv pred.", linewidth=2, linestyle=":", markersize=5, color='brown', zorder=100)

ax.errorbar(
    np.arange(2, context_len), pseudo_pred_qs[1,2:],
    yerr=[pseudo_pred_qs[1,2:] - pseudo_pred_qs[0,2:], pseudo_pred_qs[2,2:] - pseudo_pred_qs[1,2:]],
    fmt='o', color='brown',
    label=f"Pseudoinv pred.", capsize=2, markersize=5, marker=".", linestyle=":", linewidth=2, zorder=2
)

ax.set_xlabel("Context", fontsize=16)
ax.set_xlim([2, context_len])
ax.set_ylabel("Error", fontsize=16)
ax.legend(loc='upper right')
ax.set_yscale("log")
ax.set_xscale("log")
ax.grid(True)
ax.grid(which='minor', linestyle=':', linewidth=0.25, color='gray', alpha=0.5)
ax.grid(which='major', linestyle='-', linewidth=0.35, color='gray', alpha=0.5)
ax.set_ylim([2e-4, 1.2e0])

# Adding a colorbar

# After filling examples_list:
examples_array = np.array(examples_list)
# Compute boundaries: midpoints between each example count
boundaries = np.zeros(len(examples_array) + 1)
boundaries[1:-1] = (examples_array[:-1] + examples_array[1:]) / 2
# Extend the first and last boundary
boundaries[0] = examples_array[0] - (examples_array[1] - examples_array[0]) / 2
boundaries[-1] = examples_array[-1] + (examples_array[-1] - examples_array[-2]) / 2

# Select every other tick for the colorbar
tick_indices = np.arange(0, len(examples_array), 2)
cbar_ticks = examples_array[tick_indices]

norm = BoundaryNorm(boundaries, combined_cmap.N)
sm = plt.cm.ScalarMappable(cmap=combined_cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, boundaries=boundaries, ticks=cbar_ticks)
cbar.set_label("# of Training Examples", fontsize=14, rotation=270, labelpad=20)
cbar.ax.set_yticklabels([f"{int(x):.0e}" for x in cbar_ticks])  # scientific notation

plt.tight_layout()

plt.show()

timestamp = time.strftime("%y%m%d_%H")

os.makedirs("../outputs/GPT2/" + experiment_multi_train + "/figures/context", exist_ok=True)
fig.savefig("../outputs/GPT2/" + experiment_multi_train + f"/figures/context/context_plot_{timestamp}.pdf", format="pdf")


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

cmap = plt.cm.magma

colors = cmap(np.linspace(0, 1, len(np.arange(context_len))))
pseudo_colors = plt.cm.viridis(np.linspace(0, 1, 9))

for context in np.arange(0, context_len):
    train_convs = [tf_err_multi_trains[i][1][context] for i in range(len(tf_err_multi_trains))]

    ax.plot(examples_list, train_convs, label=f"_no_legend_", linewidth=2, marker=".", markersize=2, color=colors[context])
    if context < 8 and context > 1:
        skip=5
        examples_arr = np.array(examples_list)
        y = np.full(examples_arr[::skip].shape, pseudo_pred_qs[1, context], dtype=float)
        yerr_lower = np.full(examples_arr[::skip].shape, pseudo_pred_qs[1, context] - pseudo_pred_qs[0, context], dtype=float)
        yerr_upper = np.full(examples_arr[::skip].shape, pseudo_pred_qs[2, context] - pseudo_pred_qs[1, context], dtype=float)
        ax.errorbar(
            examples_arr[::skip], y,
            yerr=[yerr_lower, yerr_upper],
            fmt='o', color=pseudo_colors[context],
            label=f"Pseudoinv pred. index {context}", capsize=2, markersize=5, marker=".", linestyle=":", linewidth=2, zorder=2
        )
        # ax.hlines(y = pseudo_pred_qs[1, context], xmin=examples_list[0], xmax=examples_list[-1], color=pseudo_colors[context], linestyle='--', linewidth=2, label=f"Pseudoinv pred. index {context}")

#place a red dotted vertical line at early_stop*batch_size*gpus
early_stop = 122000
ax.axvline(x=early_stop*batch_size*gpus, color='red', linestyle='--', linewidth=2, label='Early stop checkpoint')
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("# of Training Examples", fontsize=12)
ax.set_ylabel("Error", fontsize=12)
ax.set_xlim([1e3*batch_size*gpus, max_ckpt*batch_size*gpus])
ax.grid(True)
ax.grid(which='minor', linestyle=':', linewidth='0.25', color='gray', alpha=0.5)
ax.grid(which='major', linestyle='-', linewidth='0.35', color='gray', alpha=0.5)
ax.legend(loc='lower left', fontsize=9)

#add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=5, vmax=context_len))
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Context", fontsize=12, rotation=270, labelpad=15)

plt.tight_layout()

fig_dir = "../outputs/GPT2/" + experiment_multi_train + "/figures/context"
os.makedirs(fig_dir, exist_ok=True)
timestamp = time.strftime("%y%m%d_%H%M")
fig.savefig(f"{fig_dir}/context_train_conv_plot_{timestamp}.pdf", format="pdf")

plt.show()

## zero cut train conv

In [ ]:
valA = "gaussA"
valC = "_gauss_C"
state_dim = 10
batch_size = 512 #512 #4096
gpus=2
num_val_sys = 25
experiment1 = "250114_202420.3c1184_multi_sys_trace_gaussA_state_dim_10_gauss_C_lr_1.584893192461114e-05_num_train_sys_40000" # experiment to load
experiment2 = "250127_001511.3ac954_multi_sys_trace_zero_cut_gaussA_state_dim_10_gauss_C_lr_1.584893192461114e-05_num_train_sys_40000"
datasource = "val"
nope = False
single_system = False
zero_cut = True


compute_more_ckpts = True

In [ ]:
kal_ckpts = [180000, 90000]  #60000 #81000 #28500
train_conv_plots([experiment1, experiment2], ["Multi-cut Trained", "Zero-cut Trained"], kal_ckpts, valA, valC, num_val_sys, compute_more_ckpts, 200, 1000, 180000, 1000, state_dim, single_system=single_system, nope=nope, batch_size=batch_size, gpus=gpus, zero_cut=zero_cut)